In [1]:
import numpy as np
import pandas as pd
import re
from requests import get
from bs4 import BeautifulSoup
import os
import inquirer

In [12]:
def user_input():
    url = 'https://www.iplt20.com/stats/2019'
    response = get(url)
    html_soup = BeautifulSoup(response.content, 'html.parser')
    
    years = [year.get_text() for year in html_soup.find_all('a', class_=re.compile(r'sub-menu*'))]
    if years[-1] == 'All Time Records':
        years[-1] = 'all-time'

    stats_1 = html_soup.find_all('a', class_=re.compile(r'side*'))
    stats = [re.search(r'\d/(.*)', stat['href']).group(1) for stat in stats_1]
    
    print('Pick a value for year from the following. If you want entire data then type all-time')
    for elem in years:
        print(elem)

    year_selected = None
    while year_selected not in years:
        year_selected = input("Please enter right input: ")
        
    print('\n')
        
    print("Enter a stats for which you want the data, one from the following")
    for elem in stats:
            print(elem)

    stat_selected = None
    while stat_selected not in stats:
        stat_selected = input("Please enter right input: ")
        
    return year_selected, stat_selected


In [18]:
def main_data_extraction():
    year_selected, stat_selected = user_input()
    url = 'https://www.iplt20.com/stats/'
    url_complete = url + year_selected + '/' + stat_selected
    url_complete
    response = get(url_complete)
    html_soup = BeautifulSoup(response.content, 'html.parser')
    
    column_names = re.sub(r'\n[\s]*', '\n', 
                html_soup.find('tr', class_=re.compile(r'top-players__header*')).get_text()).strip().split('\n')
    
    data_placeholder = []
    for i in html_soup.find_all('td', class_=re.compile(r'top-players*')):
            data_placeholder.append(re.sub(r'\n[\s]*', " ", i.get_text().strip()))
            
    data = np.asarray(data_placeholder).reshape(len(data_placeholder) // len(column_names), len(column_names))
    final_data = pd.DataFrame(data, columns = column_names)
    
    return final_data.head(10)

In [17]:
main_data_extraction()

Pick a value for year from the following. If you want entire data then type all-time
2019
2018
2017
2016
2015
2014
2013
2012
2011
2010
2009
2008
all-time
Please enter right input: 2019


Enter a stats for which you want the data, one from the following
most-runs
most-runs-over
most-fours
most-fours-innings
most-sixes
most-sixes-innings
most-fifties
most-centuries
fastest-fifties
fastest-centuries
highest-scores
highest-scores-innings
best-batting-strike-rate
best-batting-average
biggest-sixes
most-wickets
most-maidens
most-dot-balls
most-dot-balls-innings
best-bowling-average
best-bowling-economy
best-bowling-economy-innings
best-bowling-strike-rate
best-bowling-strike-rate-innings
best-bowling-innings
most-runs-conceded-innings
fastest-balls
most-hat-tricks
most-four-wickets
player-points
Please enter right input: most-runs


,POS,PLAYER,Mat,Inns,NO,Runs,HS,Avg,BF,SR,100,50,4s,6s
0,1,David Warner,12,12,2,692,100*,69.20,481,143.86,1,8,57,21
1,2,KL Rahul,14,14,3,593,100*,53.90,438,135.38,1,6,49,25
2,3,Quinton de Kock,16,16,1,529,81,35.26,398,132.91,0,4,45,25
3,4,Shikhar Dhawan,16,16,1,521,97*,34.73,384,135.67,0,5,64,11
4,5,Andre Russell,14,13,4,510,80*,56.66,249,204.81,0,4,31,52
